# Things to execute


In [ ]:
# graphs
import matplotlib.pyplot as plt

# animation
import matplotlib.animation as animation

# numerical methods
import numpy as np

# pytorch
import torch

# data
from torch.utils.data import Dataset, DataLoader, Subset

# neural network
import torch.nn as nn
import torch.nn.functional as F

# optimizer
import torch.optim as optim

# timer
import time

# pandas (to convert and save data)
import pandas as pd

# seaborn (heatmap)
import seaborn as sns

# google drive
from google.colab import drive
drive.mount('/content/drive')

# checkpoint
import pickle

# operating system
import os

# gaussian process regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# math tools
import math
from functools import reduce

# convolution operation
import scipy.signal

Mounted at /content/drive


In [ ]:
# MEAN & STD COMPUTER
def compute_mean_std(dataset_parameters, batch_size=16, verbose=True, normalize=False):
  """
  Function that computes the mean and standard deviation of the dataset

  """

  # create the training data and load it
  num_samples, block_size, m, n, tau, t0, rho = dataset_parameters

  # load the dataset and dataloader
  dataset = gol_dataset(num_samples, block_size, m, n, tau, t0, rho, balance=True, normalize=False)

  # turn into np
  data = np.array(dataset.data)

  # compute the mean and standard deviation
  mean = data.mean(axis=0)
  std = data.std(axis=0)

  # save it
  file_path = "/content/drive/My Drive/Colab Notebooks/"
  mean_filename = f"mean_{block_size}_{m}_{n}_{tau}_{rho}.npy"
  std_filename = f"std_{block_size}_{m}_{n}_{tau}_{rho}.npy"
  np.save(file_path + mean_filename, mean)
  np.save(file_path + std_filename, std)

  # print the results
  if verbose:
    print(f"Mean: {mean}")
    print(f"Std: {std}")


  return mean, std

def computing_neighbour_sum(grid):
  """
  Computes the neighbours' sum for each cell in the grid with periodic boundary conditions
  """
  return (
    np.roll(np.roll(grid, 1, axis=0), 1, axis=1)  # top-left neighbor
    + np.roll(np.roll(grid, 1, axis=0), 0, axis=1)  # top neighbor
    + np.roll(np.roll(grid, 1, axis=0), -1, axis=1)  # top-right neighbor
    + np.roll(np.roll(grid, 0, axis=0), 1, axis=1)  # left neighbor
    + np.roll(np.roll(grid, 0, axis=0), -1, axis=1)  # right neighbor
    + np.roll(np.roll(grid, -1, axis=0), 1, axis=1)  # bottom-left neighbor
    + np.roll(np.roll(grid, -1, axis=0), 0, axis=1)  # bottom neighbor
    + np.roll(np.roll(grid, -1, axis=0), -1, axis=1)  # bottom-right neighbor
    )

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# GAME OF LIFE NEXT STEP GENERATOR
def next_step_fn(grid):
    """Function that computes the next grid in Conway's Game of Life using periodic boundary conditions"""

    # we are shifting the grid while wrapping around at the edges (torus) so as to impose the periodic boundary conditions
    neighbours_sum = computing_neighbour_sum(grid)

    # grid's length as a variable
    n = len(grid)

    # Conway's Game of Life rules
    for i in range(n):
        for j in range(n):

            # Cells with alive elements
            if grid[i][j] == 1:
                if (neighbours_sum[i][j] <= 1) or (neighbours_sum[i][j] >= 4):
                    grid[i][j] = 0

            # Cells with dead elements
            elif grid[i][j] == 0:
                if (neighbours_sum[i][j] == 3):
                    grid[i][j] = 1

    # we return the grid
    return grid

def set_all_seeds(seed):
    # numpy seed
    np.random.seed(seed)
    # torch seed
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# COARSE GRAINING IMPLEMENTATION
def block_average(grid, block_size):
    """
    Function that applies coarse-graining for a block_size x block_size

    """
    # length parameters
    n = len(grid)

    # we compute how many blocks will there be in the coarsed grid
    new_blocks = n//block_size

    # initialize sum variables
    alive_sum = 0
    dead_sum = 0

    # initialize coarsed grid
    coarse_grid = np.zeros((new_blocks,new_blocks),dtype=int)

    # loops to go through the future blocks
    for i in range(new_blocks):

        for j in range(new_blocks):

            # we create a subgrid by slicing our original grid
            subgrid = grid[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]

            # wrap when the dimensions do not match
            # x-axis
            if subgrid.shape[0] < block_size:
              subgrid = np.vstack([subgrid, grid[0:block_size - subgrid.shape[0], j * block_size:(j + 1) * block_size]])
            # y-axis
            if subgrid.shape[1] < block_size:
              subgrid = np.hstack([subgrid, grid[i * block_size:(i + 1) * block_size, 0:block_size - subgrid.shape[1]]])

            # we sum over the elements of the subgrid
            alive_sum = np.sum(subgrid)
            dead_sum = block_size**2 - alive_sum

            # let's see if we have more alive or dead and coarse-grain as such
            if alive_sum > dead_sum:
                coarse_grid[i,j] = 1
            else:
                coarse_grid[i,j] = 0

    return coarse_grid


# MULTILAYERPERCEPTRON: CREATION OF THE NEURAL NETWORK MODEL

class MLP(nn.Module):

  def __init__(self, input_size, hidden_size, output_size, dropout_rate):
    """Initialization of the MLP

        Parameters: num_input, num_output, num_hidden
        - input_size: number of input neurons
        - output_size: number of output neurons
        - hidden_size: number of neurons in hidden layers (list because the number can differ layer to layer)

    """

    # calls the constructor of nn.Module (needed to use NN functionaliaties)
    super(MLP, self).__init__()

    # array that will be the layers
    layers = []

    # number of neurons that each layer will have (this variable will be updated each time)
    num_neurons = input_size

    # loop that creates the layers of the required size
    for num_hidden in hidden_size:

      # nn.Linear creates a fully connected layer
      layers.append(nn.Linear(num_neurons, num_hidden))

      # batch normalization
      layers.append(nn.BatchNorm1d(num_hidden))

      # activation function - ReLU: Rectified Linear Unit (f(x) = max(0,x)): used in hidden layers
      layers.append(nn.ReLU())

      # apply the Dropout
      layers.append(nn.Dropout(p=dropout_rate))

      # update the number of neurons
      num_neurons = num_hidden

    # we connect the last two matrices
    layers.append(nn.Linear(num_neurons, output_size))

    # sigmoid activation because of the binary outcome
    #layers.append(nn.Sigmoid())

    # complete model of the neural network: layers + activation functions: *unpacks the list
    self.model = nn.Sequential(*layers)

  def forward(self, x):
    """We run an input tensor (x) through the model we defined before"""
    x = x.view(x.size(0),-1)
    # we return the input tensor but with our mlp applied
    return self.model(x)

# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# L1 REGULARIZATION

def L1_regularization(model, lambda_l1):
  """
  Function that implements L1 Regularization:
    it adds a penalty to the loss function based on the abs(weights) to encourage sparsity and reduce overfitting

  Parameters:
  - model: neural network model
  - lambda_l1: parameter that controls the weight of the penalty term

  """

  # initialize the loss
  L1_loss = 0

  # add the loss corresponding to the absolute values
  for param in model.parameters():
    L1_loss += torch.sum(torch.abs(param))

  # return the loss with the weight controlling parameter
  return lambda_l1 * L1_loss

In [ ]:
# DATASET GENERATION WITH ROLLING TO FIND BETTER DATA
class gol_dataset_rolling():
  def __init__(self, num_samples, block_size, m, n, tau, t0, rho, normalize):

    """
    Initialization of the dataset: converting data into PyTorch tensors

    Parameters: num_samples, n, m, tau, block_size
    - num_samples: number of samples (random grids) that will be created
    - block_size: size of the coarsed grid (optional)
    - m: dimensions of the subgrid
    - n: dimensions of the grid
    - tau: how many generations will we generate?
    - t0: initialization time
    - rho: density
    - balance: if True, balance the dataset (equal number of alive and dead cells)
    Data: grid sequences over a tau generations
    Labels: center cell of the tau+1 generation

    """

    # number of samples
    self.num_samples = num_samples

    # grid coarsing parameter
    self.block_size = block_size

    # fixed parameters
    self.n = n
    self.m = m
    self.tau = tau

    # initialization time
    self.t0 = t0

    # density
    self.rho = rho

    # important to check whether the n and m values are possible
    assert n >= m, "n must be greater or equal than m"

    # data initialization
    self.data = []

    # labels initialization
    self.labels = []

    # generate the amount of data we want with balance
    target = num_samples//2
    alive_count = 0
    dead_count = 0
    counter = 0
    while True:

      # to add the data to the tensors we need an index counter = index
      index = 0

      if alive_count >= target and dead_count >= target:
        break

      # NxN grid
      grid = np.random.choice([0,1],(n,n),p=[1 - rho, rho])

      # we select the top-left corner from which we'll start the MxM subgrid
      i, j = np.random.randint(0, n-m+1, 2)

      # initialize sequence: matrix that will store tau grids
      sequence = np.zeros((tau, m//block_size, m//block_size), dtype=np.float32)

      # loop that iterates the initial grid t_0 times to arrive to the initialization time
      for t in range(t0):

        # we update the grid t0 generations
        grid = next_step_fn(grid)

      # loop that iterates the initial grid tau times according to the game of life rules
      for t in range(tau):

        # choose an MxM subgrid
        subgrid = grid[i:i+m, j:j+m]

        # apply coarse-graining
        coarse_grid = block_average(subgrid, block_size)

        # add to the data
        sequence[t] = coarse_grid

        # update the grid
        grid = next_step_fn(grid)

      # coarse grain the tau+1 grid
      subgrid = grid[i:i+m, j:j+m]
      coarse_grid = block_average(subgrid, block_size)
      label = coarse_grid[m//(2*block_size), m//(2*block_size)]

      if dead_count == target and label == 0:
        for i in range(len(coarse_grid)):
          for j in range(len(coarse_grid)):
            if coarse_grid[i,j] == 1:
              label = coarse_grid[i,j]
              for t in range(tau):
                sequence[t] = np.roll(sequence[t], -i+len(coarse_grid)//2, axis=0)
                sequence[t] = np.roll(sequence[t], -j+len(coarse_grid)//2, axis=1)
              break
          if label == 1:
            break

      elif alive_count == target and label == 1:
        for i in range(len(coarse_grid)):
          for j in range(len(coarse_grid)):
            if coarse_grid[i,j] == 0:
              label = coarse_grid[i,j]
              coarse_grid = np.roll(coarse_grid, -i, axis=0)
              coarse_grid = np.roll(coarse_grid, -j, axis=1)
              break
          if label == 0:
            break

      if int(label) == 1 and alive_count < target:
        self.data.append(sequence)
        self.labels.append(label)
        alive_count += 1

      elif int(label) == 0 and dead_count < target:
        self.data.append(sequence)
        self.labels.append(label)
        dead_count += 1

      counter += 1

    # print(f"Counter : {counter}, Dead count : {dead_count}, Alive count : {alive_count}")
    # data normalization
    if normalize:
      self.normalize()

    # turn into torch tensors
    self.data = torch.tensor(self.data, dtype=torch.float32)
    self.labels = torch.tensor(self.labels, dtype=torch.float32)

  def __len__(self):
    """Length of the dataset (how many samples exist)"""
    length = len(self.labels)
    return length

  def __getitem__(self,idx):
    """Retrieve a single sample from the dataset"""

    x = self.data[idx]

    # labels as the y vector
    y = self.labels[idx]

    return x,y

  def normalize(self):
    """Normalize dataset using precomputed mean and std"""

    self.data = np.array(self.data)
    tau, m, m = self.data.shape[1:]
    #mean_path = f"/content/drive/My Drive/Colab Notebooks/mean_{self.block_size}_{self.m}_{self.n}_{self.tau}_{self.rho}.npy"
    #std_path = f"/content/drive/My Drive/Colab Notebooks/std_{self.block_size}_{self.m}_{self.n}_{self.tau}_{self.rho}.npy"

    try:
      mean = np.load(mean_path)
      std = np.load(std_path)
    except:
      raise RuntimeError("Mean and std files not found")

    # avoid NaN
    eps = 1e-6
    std[std < eps] = 1
    self.data = (self.data - mean) / std


In [ ]:
# DATASET GENERATION FIXED PARAMETERS AND BLOCK SIZE AVERAGE COARSE-GRAINING
class gol_dataset():
  def __init__(self, num_samples, block_size, m, n, tau, t0, rho, balance, normalize):

    """Initialization of the dataset: converting data into PyTorch tensors

    Parameters: num_samples, n, m, tau, block_size
    - num_samples: number of samples (random grids) that will be created
    - block_size: size of the coarsed grid (optional)
    - m: dimensions of the subgrid
    - n: dimensions of the grid
    - tau: how many generations will we generate?
    - t0: initialization time
    - rho: density
    - balance: if True, balance the dataset (equal number of alive and dead cells)
    Data: grid sequences over a tau generations
    Labels: center cell of the tau+1 generation

    """

    # number of samples
    self.num_samples = num_samples

    # grid coarsing parameter
    self.block_size = block_size

    # fixed parameters
    self.n = n
    self.m = m
    self.tau = tau

    # initialization time
    self.t0 = t0

    # density
    self.rho = rho

    # important to check whether the n and m values are possible
    assert n >= m, "n must be greater or equal than m"

    # data initialization
    self.data = []

    # labels initialization
    self.labels = []

    # generate the amount of data we want with balance
    if balance:
      target = num_samples//2
      alive_count = 0
      dead_count = 0
    else:
      total_target = num_samples

    counter = 0
    while True:

      # to add the data to the tensors we need an index counter = index
      index = 0

      if balance:
        if alive_count >= target and dead_count >= target:
          break
      else:
        if len(self.data) >= total_target:
          break

      # NxN grid
      grid = np.random.choice([0,1],(n,n),p=[1 - rho, rho])

      # we select the top-left corner from which we'll start the MxM subgrid
      i, j = np.random.randint(0, n-m+1, 2)

      # initialize sequence: matrix that will store tau grids
      sequence = np.zeros((tau, m//block_size, m//block_size), dtype=np.float32)

      # loop that iterates the initial grid t_0 times to arrive to the initialization time
      for t in range(t0):

        # we update the grid t0 generations
        grid = next_step_fn(grid)

      # loop that iterates the initial grid tau times according to the game of life rules
      for t in range(tau):

        # choose an MxM subgrid
        subgrid = grid[i:i+m, j:j+m]

        # apply coarse-graining
        coarse_grid = block_average(subgrid, block_size)

        # add to the data
        sequence[t] = coarse_grid

        # update the grid
        grid = next_step_fn(grid)

      # coarse grain the tau+1 grid
      subgrid = grid[i:i+m, j:j+m]
      coarse_grid = block_average(subgrid, block_size)
      label = coarse_grid[m//(2*block_size), m//(2*block_size)]

      if balance:
        if int(label) == 1 and alive_count < target:
          self.data.append(sequence)
          self.labels.append(label)
          alive_count += 1
          index += 1
        elif int(label) == 0 and dead_count < target:
          self.data.append(sequence)
          self.labels.append(label)
          dead_count += 1
          index += 1
      else:
        self.data.append(sequence)
        self.labels.append(label)

      # have 2 datasets: don't filter one (balance and imbalanced)

      counter += 1
    print(f"Counter : {counter}, Dead count : {dead_count}, Alive coun : {alive_count}")

    if normalize:
      self.normalize()

    # normalize data
    # turn into torch tensors
    self.data = torch.tensor(self.data, dtype=torch.float32)
    self.labels = torch.tensor(self.labels, dtype=torch.float32)

  def __len__(self):
    """Length of the dataset (how many samples exist)"""
    length = len(self.labels)
    return length

  def __getitem__(self,idx):
    """Retrieve a single sample from the dataset"""

    x = self.data[idx]

    # labels as the y vector
    y = self.labels[idx]

    return x,y

  def normalize(self):
    """Normalize dataset using precomputed mean and std"""

    self.data = np.array(self.data)
    tau, m, m = self.data.shape[1:]
    mean_path = f"/content/drive/My Drive/Colab Notebooks/mean_{self.block_size}_{self.m}_{self.n}_{self.tau}_{self.rho}.npy"
    std_path = f"/content/drive/My Drive/Colab Notebooks/std_{self.block_size}_{self.m}_{self.n}_{self.tau}_{self.rho}.npy"

    try:
      mean = np.load(mean_path)
      std = np.load(std_path)
    except:
      raise RuntimeError("Mean and std files not found")

    # avoid NaN
    eps = 1e-6
    std[std < eps] = 1
    self.data = (self.data - mean) / std


# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# TRAINING AND VALIDATING
def train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed, conv, balance, lambda_l1=0.01, printing_rhythm=10, verbose=True):

  """
  Function that trains the model and validates it

  Parameters:
  - model: MLP neural network model
  - batch_size: size of the batches
  - train_size: size of the training data
  - val_size: size of the validation data
  - fixed: if True, fixed grid dimensions
          if False, multipliers
  - conv: if True, convolutional coarse graining
          if False, block average coarse graining
  - balance: if True, balance the dataset (equal number of alive and dead cells)
  - lambda_l1: parameter that controls the weight of the penalty term
  - printing_rhythm: how often to print the loss
  - dataset_parameters: parameters of the dataset (num_samples, block_size, m_multiplier, n_multiplier, tau_multiplier, t0, rho)
  - verbose: if True, prints the loss

  Returns:
  - training_loss
  - validation_loss
  - accuracy

  """

  # initialize the losses
  train_losses_per_batch = []
  val_losses_per_batch = []
  accuracy_per_batch = []

  # create the training data and load it
  num_samples, block_size, m, n, tau, t0, rho = dataset_parameters
  training_dataset = gol_dataset_rolling(train_size, block_size, m, n, tau, t0, rho, normalize=True)
  train_loader = DataLoader(training_dataset, batch_size = batch_size, shuffle=True)

  # now, we generate validation data
  validation_dataset = gol_dataset_rolling(val_size, block_size, m, n, tau, t0, rho, normalize=True)
  validation_loader = DataLoader(validation_dataset, batch_size = batch_size, shuffle=True)

  # print the length of the dataset
  print(f"Length of the training dataset: {len(training_dataset)}")

  # optimizer and loss function
  optimizer = optim.Adam(model.parameters(), lr = 1e-3)
  loss_fn = nn.BCEWithLogitsLoss()
  # loss_fn = nn.BCELoss()

  # different optimizer?
  # test different gridsizes

  # loop that goes through the batches of the training data
  for i, batch in enumerate(train_loader):

    # evaluation before training
    with torch.no_grad():
        model.eval()
        correct_before = 0
        total_before = 0
        for x_full, y_full in train_loader:
            x_full = x_full.view(x_full.size(0), -1)
            y_pred_full = model(x_full).squeeze()
            y_pred_binary_full = (y_pred_full > 0.5).float()
            correct_before += (y_pred_binary_full == y_full).float().sum().item()
            total_before += y_full.size(0)
        accuracy_before = correct_before / total_before

    # print only after printing rhythm
    if verbose and i % printing_rhythm == 0:
      print(f"[Pre-batch {i}] Accuracy on full training data: {accuracy_before:.4f}")

    # initialize the counters
    model.train()
    correct_train = 0
    total_train = 0
    zero_count_train = 0
    one_count_train = 0

    # establish training
    model.train()

    # define the x,y
    x_training, y_training = batch

    # input data
    x_training = x_training.view(x_training.size(0), -1)

    # reset gradients
    optimizer.zero_grad()

    # predict with the model (forward pass) (.squeeze erases the extradimension)
    y_pred = model(x_training).squeeze()

    # compute the training loss
    loss = loss_fn(y_pred, y_training)
    raw_loss = loss.item()

    # add L1 regularization
    loss += L1_regularization(model, lambda_l1)

    # backpropagation: compute gradient for the weights
    loss.backward()

    # to prevent exploding gradients (try vanishing gradients too)
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

    # update weights with the optimizer (Adam in our case)
    optimizer.step()

    # save the data lost by batch
    train_losses_per_batch.append(raw_loss)

    # as we get a P(0,1) we need to convert to binary values
    y_acc_train = (y_pred > 0.5).float()

    # count correct predictions
    correct_train += (y_acc_train == y_training).float().sum().item()

    # count the amount of zeros and ones
    zero_count_train += (y_acc_train == 0).float().sum().item()
    one_count_train += (y_acc_train == 1).float().sum().item()

    # count total predictions
    total_train += y_training.size(0)

    # accuracy training
    accuracy_train = correct_train / total_train

    # define the validation loss
    val_loss = 0

    # disable the gradient
    with torch.no_grad():

      correct = 0
      total = 0
      zero_count = 0
      one_count = 0
      model.eval()

      # loop that goes through the batches of data in the validation data
      for batch1 in validation_loader:

        # batch
        x_val, y_val = batch1

        # prepare input
        x_val = x_val.view(x_val.size(0), -1)

        # predict the y
        y_pred = model(x_val).squeeze()

        # as we get a P(0,1) we need to convert to binary values
        y_acc = (y_pred > 0.5).float()

        # count correct predictions
        correct += (y_acc == y_val).float().sum().item()

        # count the amount of zeros and ones
        zero_count += (y_acc == 0).float().sum().item()
        one_count += (y_acc == 1).float().sum().item()

        # count total predictions
        total += y_val.size(0)

        # compute the loss
        loss_val = loss_fn(y_pred, y_val)

        # add the loss in each step
        val_loss += loss_val.item()

      # average the validation loss for the data loaded and add it to the returned data
      val_loss /= len(validation_loader)
      val_losses_per_batch.append(val_loss)

      # compute accuracy and average loss
      accuracy = correct / total
      accuracy_per_batch.append(accuracy)

      # add convergence when it learns
      #if i >= 4 and (accuracy_per_batch[i] == accuracy_per_batch[i-1] == accuracy_per_batch[i-2] == accuracy_per_batch[i-3] == accuracy_per_batch[i-4] == 1.):
      #  print("Convergence reached: Model trained")
      #  break


      # print the values obtained if we want
      if verbose and i % printing_rhythm == 0:
        print(f"Batch: {i}, Training Loss: {raw_loss:.6f}, Validation Loss: {val_loss:.6f}, TRAINING: One counter: {int(one_count_train)}, Zero counter: {int(zero_count_train)}, Accuracy: {accuracy_train} \| VALIDATION: One counter: {int(one_count)}, Zero counter: {int(zero_count)}, Accuracy: {accuracy}")

  return model, train_losses_per_batch, val_losses_per_batch, accuracy_per_batch


# TEST THE MODEL GENERATING THE DATASET INSIDE
def testing(model, batch_size, dataset_parameters, fixed, conv, verbose=True, balance=False):

  """

  Function that tests the accuracy of the neural network

  """

  # create the training data and load it
  num_samples, block_size, m, n, tau, t0, rho = dataset_parameters
  test_dataset = gol_dataset(num_samples, block_size, m, n, tau, t0, rho, balance, normalize=True)

  # load the data and test everything
  test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False) #skip it
  loss_fn = nn.BCEWithLogitsLoss()

  # first we have to turn into evaluation mode with this command
  model.eval()

  # for faster testing, disable gradient computation
  with torch.no_grad():

    # monitorizing loss and correct predictions
    correct = 0
    total = 0
    zero_count = 0
    one_count = 0
    total_loss = 0
    real_one = 0
    real_zero = 0
    i = 1

    # loop that goes through the batches of data
    for batch in test_loader:

      # input data (x) and labels (y)
      x, y_label = batch

      # we need to flatten the input to match the dimensions when training
      x = x.view(x.size(0), -1)

      # output y_out obtained with our trained model
      y_out = model(x).squeeze()

      # as we get a P(0,1) we need to convert to binary values
      y_pred = (y_out > 0.5).float()

      # compute loss and count it
      loss = loss_fn(y_out, y_label)
      total_loss += loss.item()

      # count correct predictions
      correct += (y_pred == y_label).float().sum().item()

      # count the amount of zeros and ones
      zero_count += (y_pred == 0).float().sum().item()
      one_count += (y_pred == 1).float().sum().item()
      real_one += (y_label == 1).float().sum().item()
      real_zero += (y_label == 0).float().sum().item()

      # print(f"Batch: {i}, Prediction: {y_pred}, Real: {y_label}")

      # count total predictions
      total += y_label.size(0)

      i+=1
  # compute accuracy and average loss
  accuracy = correct / total
  avg_loss = total_loss / len(test_loader)

  # print the results
  if verbose:
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Predicted one counter: {int(one_count)}, Predicted zero counter: {int(zero_count)}")
    print(f"Real one counter: {int(real_one)}, Real zero counter: {int(real_zero)}")

  # return accuracy and loss for the dataset
  return accuracy, avg_loss


In [ ]:
# rolling dataset
set_all_seeds(73)
dataset = gol_dataset_rolling(1000, 5, 15, 15, 5, 0, 0.5, False)
set_all_seeds(73)
dataset_2 = gol_dataset(1000, 5, 15, 15, 5, 0, 0.5, True, False)

Counter : 2736, Dead count : 500, Alive count : 500
Counter : 17924, Dead count : 500, Alive coun : 500


# Tests

In [ ]:
# set the seed
seed = 73
set_all_seeds(seed)

In [ ]:
# compute mean & std
for block_size in range(7, 12, 2):
  num_samples = 15000
  n = 25
  m = 25
  rho = 0.5
  t0 = 0
  tau = block_size
  dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
  # obtain mean & std
  mean, std = compute_mean_std(dataset_parameters)

## Block size = 1 (wrong normalization)

In [ ]:
#fix n,m and iterate through different block_sizes
n = 15
m = 15
block_size = 1
print("_______________________________________________________________________")
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 0
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [input_size//2, input_size//4]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)

_______________________________________________________________________
Block size = 1
Model created with input_size: 225
Starting training...
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.729724, Validation Loss: 0.695159, TRAINING: One counter: 0, Zero counter: 16, Accuracy: 0.5625 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.5073
Batch: 10, Training Loss: 0.736712, Validation Loss: 0.680801, TRAINING: One counter: 5, Zero counter: 11, Accuracy: 0.5625 \| VALIDATION: One counter: 15, Zero counter: 1009, Accuracy: 0.5009765625
[Pre-batch 20] Accuracy on full training data: 0.5000
Batch: 20, Training Loss: 0.642274, Validation Loss: 0.671909, TRAINING: One counter: 3, Zero counter: 13, Accuracy: 0.8125 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 30] Accuracy on full training data: 0.5000
Batch: 30, Training Loss: 0.

In [ ]:
#__________________________________________________________________________________________________________________________________________
# IMBALANCED DATASET
n = 15
m = 15
block_size = 1
tau = block_size
rho = 0.5
correct = 0
total = 0
one = 0
zero = 0
ones = 0
zeros = 0
mean_path = f"/content/drive/My Drive/Colab Notebooks/mean_{block_size}_{m}_{n}_{tau}_{rho}.npy"
std_path = f"/content/drive/My Drive/Colab Notebooks/std_{block_size}_{m}_{n}_{tau}_{rho}.npy"
mean = np.load(mean_path)
std = np.load(std_path)
mean = torch.tensor(mean, dtype=torch.float32)
std = torch.tensor(std, dtype=torch.float32)
batch_x = []
batch_y_true = []
batch_size = 16
total_samples = 1789
for iter in range(total_samples):
    trained_model_test_1.eval()
    with torch.no_grad():
        grid = np.random.choice([0,1],(n,n),p=[1 - rho, rho])
        x_single_sample = []
        for _ in range(tau):
            coarse_grid = block_average(grid, block_size)
            x_single_sample.append(coarse_grid)
            grid = next_step_fn(grid)
        x_single_sample = np.array(x_single_sample)
        coarse_grid_final = block_average(grid, block_size)
        y_single_sample = coarse_grid_final[m//(2*block_size), m//(2*block_size)]
        x_single_sample = torch.tensor(x_single_sample, dtype=torch.float32)
        # normalize
        x_single_sample = (x_single_sample - mean) / std
        eps = 1e-6
        std[std < eps] = 1.0
        x_single_sample = x_single_sample.view(1, -1)
        batch_x.append(x_single_sample)
        batch_y_true.append(y_single_sample)
        # process the batch when it's full or at the end of iterations
        if len(batch_x) == batch_size or (iter == (1000 - 1) and len(batch_x) > 0):
            x_batch_tensor = torch.cat(batch_x, dim=0)
            y_true_batch_tensor = torch.tensor(batch_y_true, dtype=torch.float32)

            y_pred_batch = trained_model_test(x_batch_tensor).squeeze()

            # in case it is only a scalar
            if y_pred_batch.dim() == 0:
                y_pred_batch = y_pred_batch.unsqueeze(0)

            # prediction batch
            y_pred_batch = (y_pred_batch > 0.5).float()

            # accuracies
            for i in range(len(batch_x)):
                y_pred = y_pred_batch[i]
                y_true = y_true_batch_tensor[i]

                if y_pred == 0:
                    zero += 1
                elif y_pred == 1:
                    one += 1
                if y_true == 0:
                    zeros += 1
                elif y_true == 1:
                    ones += 1
                correct += (y_pred == y_true).float().sum().item()
                total += 1

            # reset batch for the next iteration
            batch_x = []
            batch_y_true = []

print(f"Imbalanced dataset: Accuracy: {correct/total}, Predicted 0s: {zero}, Predicted 1s: {one}, Real 0s: {zeros}, Real 1s: {ones}")

#__________________________________________________________________________________________________________________________________________
# BALANCED AND RAW DATASET
num_samples = 1000
balanced_data = []
balanced_labels = []
raw_data = []
raw_labels = []
alive_count = 0
dead_count = 0
target = num_samples//2
while True:
  grid = np.random.choice([0,1],(n,n),p=[1 - rho, rho])
  sequence = np.zeros((tau, m//block_size, m//block_size), dtype=np.float32)
  for t in range(tau):
    coarse_grid = block_average(grid, block_size)
    sequence[t] = coarse_grid
    grid = next_step_fn(grid)
  coarse_grid = block_average(grid, block_size)
  label = coarse_grid[m//(2*block_size), m//(2*block_size)]
  raw_data.append(sequence)
  raw_labels.append(label)
  if int(label) == 1 and alive_count < target:
    balanced_data.append(sequence)
    balanced_labels.append(label)
    alive_count += 1
  elif int(label) == 0 and dead_count < target:
    balanced_data.append(sequence)
    balanced_labels.append(label)
    dead_count += 1
  if alive_count == target and dead_count == target:
    break
balanced_data = np.array(balanced_data)
raw_data = np.array(raw_data)
mean_path = f"/content/drive/My Drive/Colab Notebooks/mean_{block_size}_{m}_{n}_{tau}_{rho}.npy"
std_path = f"/content/drive/My Drive/Colab Notebooks/std_{block_size}_{m}_{n}_{tau}_{rho}.npy"
mean = np.load(mean_path)
std = np.load(std_path)
eps = 1e-6
std[std<eps] = 1
balanced_data = (balanced_data - mean) / std
raw_data = (raw_data - mean) / std
balanced_data = torch.tensor(balanced_data, dtype=torch.float32)
raw_data = torch.tensor(raw_data, dtype=torch.float32)
balanced_labels = torch.tensor(balanced_labels, dtype=torch.float32)
raw_labels = torch.tensor(raw_labels, dtype=torch.float32)
print(f"Amount of raw data: {len(raw_data)}")
print(f"Amount of balanced data: {len(balanced_data)}")
z = 0
o = 0
for label in balanced_labels:
  if label== 0:
    z += 1
  elif label == 1:
    o += 1
print(f"Balanced dataset: 0s: {z}, 1s: {o}")

# predictions
trained_model_test_1.eval()
with torch.no_grad():

  # balanced data
  zero = 0
  one = 0
  zeros = 0
  ones = 0
  correct = 0
  total = 0
  for i in range(len(balanced_data)):
    x = balanced_data[i]
    y_true = balanced_labels[i]
    x = x.view(1, -1)
    y_pred = trained_model_test(x).squeeze()
    y_pred = (y_pred > 0.5).float()
    correct += (y_pred == y_true).float().sum().item()
    total += 1
    if y_pred == 0:
      zero += 1
    elif y_pred == 1:
      one += 1
    if y_true == 0:
      zeros += 1
    elif y_true == 1:
      ones += 1

  print(f"Balanced dataset: Accuracy: {correct/total}, Predicted 0s: {zero}, Predicted 1s: {one}, Real 0s: {zeros}, Real 1s: {ones}")

  # raw data
  zero = 0
  one = 0
  zeros = 0
  ones = 0
  correct = 0
  total = 0
  for i in range(len(raw_data)):
    x = raw_data[i]
    y_true = raw_labels[i]
    x = x.view(1, -1)
    y_pred = trained_model_test(x).squeeze()
    y_pred = (y_pred > 0.5).float()
    correct += (y_pred == y_true).float().sum().item()
    total += 1
    if y_pred == 0:
      zero += 1
    elif y_pred == 1:
      one += 1
    if y_true == 0:
      zeros += 1
    elif y_true == 1:
      ones += 1

  print(f"Raw dataset: Accuracy: {correct/total}, Predicted 0s: {zero}, Predicted 1s: {one}, Real 0s: {zeros}, Real 1s: {ones}")

Imbalanced dataset: Accuracy: 0.8923766816143498, Predicted 0s: 1126, Predicted 1s: 658, Real 0s: 1302, Real 1s: 482
Amount of raw data: 1792
Amount of balanced data: 1000
Balanced dataset: 0s: 500, 1s: 500
Balanced dataset: Accuracy: 0.92, Predicted 0s: 448, Predicted 1s: 552, Real 0s: 500, Real 1s: 500
Raw dataset: Accuracy: 0.8934151785714286, Predicted 0s: 1129, Predicted 1s: 663, Real 0s: 1292, Real 1s: 500


In [ ]:
# try using the testing function now
testing(trained_model_test, batch_size=16, dataset_parameters=dataset_parameters, fixed=False, conv=False, verbose=True, balance=True)

========DATASET=========
Batch: 1, Prediction: tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.]), Real: tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0.])
Batch: 2, Prediction: tensor([0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1.]), Real: tensor([0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1.])
Batch: 3, Prediction: tensor([0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.]), Real: tensor([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.])
Batch: 4, Prediction: tensor([0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.]), Real: tensor([0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.])
Batch: 5, Prediction: tensor([0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.]), Real: tensor([0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
Batch: 6, Prediction: tensor([1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.]), Real

(0.9017857142857143, 0.0)

## Block size = 1 (good normalization)

In [ ]:
#fix n,m and iterate through different block_sizes
n = 15
m = 15
block_size = 1
print("_______________________________________________________________________")
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 0
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [input_size//2, input_size//4]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)

_______________________________________________________________________
Block size = 1
Model created with input_size: 225
Starting training...
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.770668, Validation Loss: 0.695594, TRAINING: One counter: 0, Zero counter: 16, Accuracy: 0.375 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.5007
Batch: 10, Training Loss: 0.784085, Validation Loss: 0.681388, TRAINING: One counter: 4, Zero counter: 12, Accuracy: 0.5625 \| VALIDATION: One counter: 2, Zero counter: 1022, Accuracy: 0.501953125
[Pre-batch 20] Accuracy on full training data: 0.5010
Batch: 20, Training Loss: 0.659027, Validation Loss: 0.675190, TRAINING: One counter: 2, Zero counter: 14, Accuracy: 0.6875 \| VALIDATION: One counter: 3, Zero counter: 1021, Accuracy: 0.5029296875
[Pre-batch 30] Accuracy on full training data: 0.5000
Batch: 30, Training Lo

In [ ]:
#fix n,m and iterate through different block_sizes

# n = 25
# m = 25
n = 25
m = 25
block_size = 1
print("_______________________________________________________________________")
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 0
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [input_size//2, input_size//4]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)

_______________________________________________________________________
Block size = 1
Model created with input_size: 625
Starting training...
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.740449, Validation Loss: 0.694549, TRAINING: One counter: 1, Zero counter: 15, Accuracy: 0.5625 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.5000
Batch: 10, Training Loss: 0.641907, Validation Loss: 0.691980, TRAINING: One counter: 2, Zero counter: 14, Accuracy: 0.5625 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 20] Accuracy on full training data: 0.5002
Batch: 20, Training Loss: 0.797815, Validation Loss: 0.688128, TRAINING: One counter: 5, Zero counter: 11, Accuracy: 0.4375 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 30] Accuracy on full training data: 0.5051
Batch: 30, Training Loss: 0.621975, Va

In [ ]:
# save the trained_model_test in my google drive
torch.save(trained_model_test.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_1.pth")

In [ ]:
# retrieve the saved model
input_size = 225
hidden_size = [input_size//2, input_size//4]
output_size = 1
trained_model_test_1 = MLP(input_size, hidden_size, output_size, dropout_rate=0)
trained_model_test_1.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/trained_model_test_1.pth"))
print(trained_model_test)

MLP(
  (model): Sequential(
    (0): Linear(in_features=225, out_features=112, bias=True)
    (1): BatchNorm1d(112, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0, inplace=False)
    (4): Linear(in_features=112, out_features=56, bias=True)
    (5): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0, inplace=False)
    (8): Linear(in_features=56, out_features=1, bias=True)
  )
)


In [ ]:
#__________________________________________________________________________________________________________________________________________
# IMBALANCED DATASET
n = 15
m = 15
block_size = 1
tau = block_size
rho = 0.5
correct = 0
total = 0
one = 0
zero = 0
ones = 0
zeros = 0
mean_path = f"/content/drive/My Drive/Colab Notebooks/mean_{block_size}_{m}_{n}_{tau}_{rho}.npy"
std_path = f"/content/drive/My Drive/Colab Notebooks/std_{block_size}_{m}_{n}_{tau}_{rho}.npy"
mean = np.load(mean_path)
std = np.load(std_path)
mean = torch.tensor(mean, dtype=torch.float32)
std = torch.tensor(std, dtype=torch.float32)
batch_x = []
batch_y_true = []
batch_size = 16
total_samples = 1789
for iter in range(total_samples):
    trained_model_test_1.eval()
    with torch.no_grad():
        grid = np.random.choice([0,1],(n,n),p=[1 - rho, rho])
        x_single_sample = []
        for _ in range(tau):
            coarse_grid = block_average(grid, block_size)
            x_single_sample.append(coarse_grid)
            grid = next_step_fn(grid)
        x_single_sample = np.array(x_single_sample)
        coarse_grid_final = block_average(grid, block_size)
        y_single_sample = coarse_grid_final[m//(2*block_size), m//(2*block_size)]
        x_single_sample = torch.tensor(x_single_sample, dtype=torch.float32)
        # normalize
        x_single_sample = (x_single_sample - mean) / std
        eps = 1e-6
        std[std < eps] = 1.0
        x_single_sample = x_single_sample.view(1, -1)
        batch_x.append(x_single_sample)
        batch_y_true.append(y_single_sample)
        # process the batch when it's full or at the end of iterations
        if len(batch_x) == batch_size or (iter == (1000 - 1) and len(batch_x) > 0):
            x_batch_tensor = torch.cat(batch_x, dim=0)
            y_true_batch_tensor = torch.tensor(batch_y_true, dtype=torch.float32)

            y_pred_batch = trained_model_test(x_batch_tensor).squeeze()

            # in case it is only a scalar
            if y_pred_batch.dim() == 0:
                y_pred_batch = y_pred_batch.unsqueeze(0)

            # prediction batch
            y_pred_batch = (y_pred_batch > 0.5).float()

            # accuracies
            for i in range(len(batch_x)):
                y_pred = y_pred_batch[i]
                y_true = y_true_batch_tensor[i]

                if y_pred == 0:
                    zero += 1
                elif y_pred == 1:
                    one += 1
                if y_true == 0:
                    zeros += 1
                elif y_true == 1:
                    ones += 1
                correct += (y_pred == y_true).float().sum().item()
                total += 1

            # reset batch for the next iteration
            batch_x = []
            batch_y_true = []

print(f"Imbalanced dataset: Accuracy: {correct/total}, Predicted 0s: {zero}, Predicted 1s: {one}, Real 0s: {zeros}, Real 1s: {ones}")

#__________________________________________________________________________________________________________________________________________
# BALANCED AND RAW DATASET
num_samples = 1000
balanced_data = []
balanced_labels = []
raw_data = []
raw_labels = []
alive_count = 0
dead_count = 0
target = num_samples//2
while True:
  grid = np.random.choice([0,1],(n,n),p=[1 - rho, rho])
  sequence = np.zeros((tau, m//block_size, m//block_size), dtype=np.float32)
  for t in range(tau):
    coarse_grid = block_average(grid, block_size)
    sequence[t] = coarse_grid
    grid = next_step_fn(grid)
  coarse_grid = block_average(grid, block_size)
  label = coarse_grid[m//(2*block_size), m//(2*block_size)]
  raw_data.append(sequence)
  raw_labels.append(label)
  if int(label) == 1 and alive_count < target:
    balanced_data.append(sequence)
    balanced_labels.append(label)
    alive_count += 1
  elif int(label) == 0 and dead_count < target:
    balanced_data.append(sequence)
    balanced_labels.append(label)
    dead_count += 1
  if alive_count == target and dead_count == target:
    break
balanced_data = np.array(balanced_data)
raw_data = np.array(raw_data)
mean_path = f"/content/drive/My Drive/Colab Notebooks/mean_{block_size}_{m}_{n}_{tau}_{rho}.npy"
std_path = f"/content/drive/My Drive/Colab Notebooks/std_{block_size}_{m}_{n}_{tau}_{rho}.npy"
mean = np.load(mean_path)
std = np.load(std_path)
eps = 1e-6
std[std<eps] = 1
balanced_data = (balanced_data - mean) / std
raw_data = (raw_data - mean) / std
balanced_data = torch.tensor(balanced_data, dtype=torch.float32)
raw_data = torch.tensor(raw_data, dtype=torch.float32)
balanced_labels = torch.tensor(balanced_labels, dtype=torch.float32)
raw_labels = torch.tensor(raw_labels, dtype=torch.float32)
print(f"Amount of raw data: {len(raw_data)}")
print(f"Amount of balanced data: {len(balanced_data)}")
z = 0
o = 0
for label in balanced_labels:
  if label== 0:
    z += 1
  elif label == 1:
    o += 1
print(f"Balanced dataset: 0s: {z}, 1s: {o}")

# predictions
trained_model_test_1.eval()
with torch.no_grad():

  # balanced data
  zero = 0
  one = 0
  zeros = 0
  ones = 0
  correct = 0
  total = 0
  for i in range(len(balanced_data)):
    x = balanced_data[i]
    y_true = balanced_labels[i]
    x = x.view(1, -1)
    y_pred = trained_model_test(x).squeeze()
    y_pred = (y_pred > 0.5).float()
    correct += (y_pred == y_true).float().sum().item()
    total += 1
    if y_pred == 0:
      zero += 1
    elif y_pred == 1:
      one += 1
    if y_true == 0:
      zeros += 1
    elif y_true == 1:
      ones += 1

  print(f"Balanced dataset: Accuracy: {correct/total}, Predicted 0s: {zero}, Predicted 1s: {one}, Real 0s: {zeros}, Real 1s: {ones}")

  # raw data
  zero = 0
  one = 0
  zeros = 0
  ones = 0
  correct = 0
  total = 0
  for i in range(len(raw_data)):
    x = raw_data[i]
    y_true = raw_labels[i]
    x = x.view(1, -1)
    y_pred = trained_model_test(x).squeeze()
    y_pred = (y_pred > 0.5).float()
    correct += (y_pred == y_true).float().sum().item()
    total += 1
    if y_pred == 0:
      zero += 1
    elif y_pred == 1:
      one += 1
    if y_true == 0:
      zeros += 1
    elif y_true == 1:
      ones += 1

  print(f"Raw dataset: Accuracy: {correct/total}, Predicted 0s: {zero}, Predicted 1s: {one}, Real 0s: {zeros}, Real 1s: {ones}")

Imbalanced dataset: Accuracy: 0.8727578475336323, Predicted 0s: 1025, Predicted 1s: 759, Real 0s: 1242, Real 1s: 542
Amount of raw data: 1840
Amount of balanced data: 1000
Balanced dataset: 0s: 500, 1s: 500
Balanced dataset: Accuracy: 0.928, Predicted 0s: 446, Predicted 1s: 554, Real 0s: 500, Real 1s: 500
Raw dataset: Accuracy: 0.8847826086956522, Predicted 0s: 1146, Predicted 1s: 694, Real 0s: 1340, Real 1s: 500


In [ ]:
# try using the testing function now
testing(trained_model_test_1, batch_size=16, dataset_parameters=dataset_parameters, fixed=False, conv=False, verbose=True, balance=True)

========DATASET=========
Batch: 1, Prediction: tensor([1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.]), Real: tensor([1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.])
Batch: 2, Prediction: tensor([1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]), Real: tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.])
Batch: 3, Prediction: tensor([0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.]), Real: tensor([0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
Batch: 4, Prediction: tensor([0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.]), Real: tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.])
Batch: 5, Prediction: tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]), Real: tensor([0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Batch: 6, Prediction: tensor([1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.]), Real

(0.8794642857142857, 0.0)

## Block size = 3

In [ ]:
# LR = 1e-4
# dropout = 0
# n = 15
# m = 15
# hidden_size = [128, 64]

print("_______________________________________________________________________")
block_size = 3
n = 15
m = 15
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 0
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [128, 64]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test_3, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
# save the trained_model_test in my google drive
torch.save(trained_model_test_3.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_3.pth")

# 55% final accuracy

_______________________________________________________________________
Block size = 3
Model created with input_size: 75
Starting training...


KeyboardInterrupt: 

In [ ]:
# LR = 1e-2
# dropout = 0
# n = 15
# m = 15
# hidden_size = [128, 64]
# t0 = 15
print("_______________________________________________________________________")
block_size = 3
n = 15
m = 15
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 15
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [128, 64]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test_3, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
# save the trained_model_test in my google drive
torch.save(trained_model_test_3.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_3.pth")

_______________________________________________________________________
Block size = 3
Model created with input_size: 75
Starting training...
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.639894, Validation Loss: 0.696307, TRAINING: One counter: 5, Zero counter: 11, Accuracy: 0.6875 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.4999
Batch: 10, Training Loss: 0.743087, Validation Loss: 0.691035, TRAINING: One counter: 5, Zero counter: 11, Accuracy: 0.625 \| VALIDATION: One counter: 4, Zero counter: 1020, Accuracy: 0.501953125
[Pre-batch 20] Accuracy on full training data: 0.5059
Batch: 20, Training Loss: 0.729026, Validation Loss: 0.682842, TRAINING: One counter: 3, Zero counter: 13, Accuracy: 0.4375 \| VALIDATION: One counter: 18, Zero counter: 1006, Accuracy: 0.509765625
[Pre-batch 30] Accuracy on full training data: 0.5284
Batch: 30, Training Los

In [ ]:
# try it with the testing function
testing(trained_model_test_3, batch_size=16, dataset_parameters=dataset_parameters, fixed=False, conv=False, verbose=True, balance=True)

========DATASET=========
Batch: 1, Prediction: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), Real: tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Batch: 2, Prediction: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), Real: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Batch: 3, Prediction: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), Real: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Batch: 4, Prediction: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), Real: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
Batch: 5, Prediction: tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), Real: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.])
Batch: 6, Prediction: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), Real

(0.9285714285714286, 0.0)

In [ ]:
# LR = 1e-2
# dropout = 0
# n = 15
# m = 15
# hidden_size = [128, 64, 32, 16, 8]
# t0 = 15
print("_______________________________________________________________________")
block_size = 3
n = 15
m = 15
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 15
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [128, 64, 32, 16, 8]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test_3, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
# save the trained_model_test in my google drive
torch.save(trained_model_test_3.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_3.pth")

_______________________________________________________________________
Block size = 3
Model created with input_size: 75
Starting training...
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.870932, Validation Loss: 0.706748, TRAINING: One counter: 10, Zero counter: 6, Accuracy: 0.4375 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.4999
Batch: 10, Training Loss: 0.717119, Validation Loss: 0.710818, TRAINING: One counter: 11, Zero counter: 5, Accuracy: 0.5 \| VALIDATION: One counter: 1, Zero counter: 1023, Accuracy: 0.5009765625
[Pre-batch 20] Accuracy on full training data: 0.5285
Batch: 20, Training Loss: 0.721507, Validation Loss: 0.712069, TRAINING: One counter: 12, Zero counter: 4, Accuracy: 0.5 \| VALIDATION: One counter: 249, Zero counter: 775, Accuracy: 0.5322265625
[Pre-batch 30] Accuracy on full training data: 0.5585
Batch: 30, Training Loss: 

In [ ]:
# LR = 1e-2
# dropout = 0
# n = 25
# m = 25
# hidden_size = [128, 64]
# t0 = 0
print("_______________________________________________________________________")
block_size = 3
n = 25
m = 25
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 0
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [128, 64]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test_3, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
# save the trained_model_test in my google drive
torch.save(trained_model_test_3.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_3.pth")

_______________________________________________________________________
Block size = 3
Model created with input_size: 192
Starting training...
Counter : 16002, Dead count : 8000, Alive count : 8000
Counter : 1024, Dead count : 512, Alive count : 512
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.661995, Validation Loss: 0.696084, TRAINING: One counter: 1, Zero counter: 15, Accuracy: 0.5625 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.4999
Batch: 10, Training Loss: 0.703307, Validation Loss: 0.696651, TRAINING: One counter: 0, Zero counter: 16, Accuracy: 0.375 \| VALIDATION: One counter: 1, Zero counter: 1023, Accuracy: 0.4990234375
[Pre-batch 20] Accuracy on full training data: 0.4992
Batch: 20, Training Loss: 0.683456, Validation Loss: 0.698541, TRAINING: One counter: 3, Zero counter: 13, Accuracy: 0.6875 \| VALIDATION: One counter: 5, Zero counte

## Block size = 5

In [ ]:
# LR = 1e-2
# dropout = 0
# n = 15
# m = 15
# hidden_size = [128, 64]
# t0 = 15
print("_______________________________________________________________________")
block_size = 5
n = 15
m = 15
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 15
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [128, 64]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test_5, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
# save the trained_model_test in my google drive
torch.save(trained_model_test_5.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_5.pth")

_______________________________________________________________________
Block size = 5
Model created with input_size: 45
Starting training...
Length of the training dataset: 16000
[Pre-batch 0] Accuracy on full training data: 0.5000
Batch: 0, Training Loss: 0.613519, Validation Loss: 0.693307, TRAINING: One counter: 3, Zero counter: 13, Accuracy: 0.625 \| VALIDATION: One counter: 0, Zero counter: 1024, Accuracy: 0.5
[Pre-batch 10] Accuracy on full training data: 0.5035
Batch: 10, Training Loss: 0.685197, Validation Loss: 0.693922, TRAINING: One counter: 3, Zero counter: 13, Accuracy: 0.5625 \| VALIDATION: One counter: 19, Zero counter: 1005, Accuracy: 0.5048828125
[Pre-batch 20] Accuracy on full training data: 0.5147
Batch: 20, Training Loss: 0.741964, Validation Loss: 0.690903, TRAINING: One counter: 3, Zero counter: 13, Accuracy: 0.5 \| VALIDATION: One counter: 69, Zero counter: 955, Accuracy: 0.5087890625
[Pre-batch 30] Accuracy on full training data: 0.5337
Batch: 30, Training Loss

In [ ]:
# LR = 1e-2
# dropout = 0
# n = 25
# m = 25
# hidden_size = [128, 64]
# t0 = 0
print("_______________________________________________________________________")
block_size = 5
n = 25
m = 25
print(f"Block size = {block_size}")
num_samples = 2**10
t0 = 15
rho = 0.5
tau = block_size
dataset_parameters = [num_samples, block_size, m, n, tau, t0, rho]
batch_size = 16
train_size = 16*1000
val_size = 2**10
input_size = (m//block_size)**2 * (tau)
hidden_size = [128, 64]
output_size = 1
model = MLP(input_size, hidden_size, output_size, dropout_rate=0)
print(f"Model created with input_size: {input_size}")
print("Starting training...")
trained_model_test_5, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
# save the trained_model_test in my google drive
torch.save(trained_model_test_5.state_dict(), "/content/drive/My Drive/Colab Notebooks/trained_model_test_5.pth")

_______________________________________________________________________
Block size = 5
Model created with input_size: 125
Starting training...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-1869367598>", line 26, in <cell line: 0>
    trained_model_test_5, train_losses, val_losses, acc_batch = train_validate(model, batch_size, train_size, val_size, dataset_parameters, fixed=True, conv=False, balance=True, lambda_l1=0.01, printing_rhythm = 10, verbose=True)
                                                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-3-2349955934>", line 202, in train_validate
    training_dataset = gol_dataset(train_size, block_size, m, n, tau, t0, rho, balance=True, normalize=True)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^